In [ ]:
# Magic word does not work for specific tensorflow version
# %tensorflow_version 2.4

# But this works
!pip install tensorflow-gpu==2.4.1
#!pip install matplotlib==3.5.3
!pip install numpy==1.19.5
!pip install opencv==4.5.1.48

# Hack to make colab working (Env restart required)
!pip uninstall -y matplotlib
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████▊                   | 156.7 MB 3.3 MB/s eta 0:01:12

In [1]:
# Mount Google Drive to /content/gdrive dir
from google.colab import drive
drive.mount('/content/gdrive')

# List the files in the My Drive dir
!ls "/content/gdrive/My Drive/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 1epocha-weights.h5    50eWeights.h5	  inputs	      segmentations.zip
 50epocha-weights.h5  'Colab Notebooks'  'Machine Learning'


In [ ]:
# Imports
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

autoencoder = tf.keras.models.load_model('/content/gdrive/My Drive/50eWeights.h5')
#autoencoder.summary()

# A quick Tensorflow Data loader
class MyInputsDataLoader:
    def __init__(self, inputs_dir, target_shape=(512, 512)):
        inputs_dir = self.__fix_glob(inputs_dir)
        self.files = tf.data.Dataset.list_files(inputs_dir, shuffle=True)
        self.AUTOTUNE = tf.data.experimental.AUTOTUNE
        self.target_shape = target_shape
        
    def __fix_glob(self, dir_path):
        if not dir_path.endswith("*"):
            dir_path = os.path.join(dir_path, "*")
            
        return dir_path
    
    @tf.function
    def parse_images(self, image_path):
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32)
        
        image = tf.image.resize(
            image, 
            self.target_shape, 
        ) / 255.
        
        return image
    
    @tf.function
    def data_generator(self, batch_size=10):
        dataset = self.files.map(self.parse_images)
        dataset = dataset.repeat()
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size = self.AUTOTUNE)
        return dataset
    
input_dataset = MyInputsDataLoader('/content/gdrive/My Drive/inputs/*')
batch = input_dataset.data_generator(20)
inputs = next(iter(batch))
print(f"Input Shape: {inputs.shape}")


preds = autoencoder.predict(inputs)

plt.rcParams["figure.figsize"] = (20, 5)
threshold = 0.5

for i, p in zip(inputs, preds):
    op = p.copy()
    p[p < threshold] = 0
    p[p >= threshold] = 1
    
    plt.subplot(1, 3, 1)
    plt.imshow(i)
    plt.title("Input Image")
    plt.axis("off")
    plt.subplot(1, 3, 2)
    plt.imshow(op, cmap="gray")
    plt.title("Predicted Image")
    plt.axis("off")
    plt.subplot(1, 3, 3)
    plt.imshow(p, cmap="gray")
    plt.title("Predicted Mask (Thresholded)")
    plt.axis("off")

plt.show()